直接用二分类的loss去做，会涉及到阈值的问题不是很划算

In [ ]:

from torch.utils.data import DataLoader,random_split
from torch.utils.data import Dataset
from transformers import BertTokenizer, AdamW,AutoModel, AutoTokenizer
import numpy as np 
import random
import json
from tqdm import tqdm
import pandas as pd


from torch.utils.data import Dataset
import torch

from transformers import BertTokenizer, AdamW
from torch.utils.data import DataLoader,random_split


In [ ]:
debug = False
if debug :
    epochs = 2
    train_batch = 16
    valid_batch = 4
else :
    epochs = 20
    train_batch = 16
    valid_batch = 64

learning_rate = 2e-5



In [ ]:
model_root_path = 'E:/pre-train-model' # model root path on windows


model_list = [
    'bert-base-chinese',
    'chinese-bert-wwm',
    
]
model_idx =0
model_path = model_root_path + '/'+model_list[model_idx]+'/'
# model download from hugging-face
model_path = 'bert-base-chinese'
tokenizer = BertTokenizer.from_pretrained(model_path)
# tokenizer = AutoTokenizer.from_pretrained("hfl/chinese-roberta-wwm-ext")
# model_path = "thunlp/Lawformer"

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:

process_data_path = '../preprocessdata/'
data = pd.read_csv(process_data_path+'train.csv')

# drop nan text 
data=data[data["content"].notnull()]

vec_frame = pd.read_csv(process_data_path+'vec_frame.csv')
if debug:
    vec_frame = vec_frame.head(200)
train_rate = 0.85
case_size = vec_frame.shape[0]


train_index_list = vec_frame.iloc[0:int(train_rate*case_size),0]
valid_index_list = vec_frame.iloc[int(train_rate*case_size):,0]


train_label = vec_frame[vec_frame["0"].isin(train_index_list)]
valid_label = vec_frame[vec_frame["0"].isin(valid_index_list)]
train_data = data[data["id"].isin(train_index_list)]
valid_data = data[data["id"].isin(valid_index_list)]

num = train_data.shape[0]
p_num = train_data[train_data["label"]!= '234'].shape[0]
n_num = train_data[train_data["label"]== '234'].shape[0]
print('样本数:',num)
print('正样本数:',p_num)
print('负样本数:',n_num)
print('正负样本比例:',p_num/n_num)
pn_rate = 5
if pn_rate<p_num/n_num:
    n_change_num = n_num
else:
    n_change_num = int(p_num/pn_rate)

n_train_data=train_data[train_data["label"]=='234'].sample(n=n_change_num,replace=True)
p_train_data=train_data[train_data["label"]!='234']
train_data_now = pd.concat([n_train_data,p_train_data],axis=0)

print('训练样本数：',train_data_now.shape)

In [ ]:
# c = []
# for i in train_data_now["content"]:
#     c.append(len(i))
# import matplotlib.pyplot as plt
# plt.hist(c)

In [ ]:

class CaseData(Dataset):
    def __init__(self, data,index_list,class_num):
        self.data = data
        self.index_list = index_list
        self.class_num = class_num
 

    def __getitem__(self, idx):
        fact = self.data.iloc[idx,2]
        id = int(self.data.iloc[idx,0])
        label_list = self.data.iloc[idx,1]
        label = torch.zeros(self.class_num)
        for i in label_list.split("#"):
            if int(i) != 234:
                label[int(i)] = 1
        return id,fact, label

    def __len__(self):
        return len(self.data)

class_num = 234
train_dataset = CaseData(train_data_now,train_index_list,class_num=class_num)
valid_dataset = CaseData(valid_data,valid_index_list,class_num=class_num)

# print(len(full_data))
train_dataloader = DataLoader(train_dataset, batch_size=train_batch, shuffle=True)
valid_dataloader = DataLoader(valid_dataset,batch_size=valid_batch,shuffle=False)


In [ ]:

def multilabel_cross_entropy(y_pred, y_true):
    """
    https://kexue.fm/archives/7359
    """
    y_pred = (1 - 2 * y_true) * y_pred  # -1 -> pos classes, 1 -> neg classes
    y_pred_neg = y_pred - y_true * 1e12  # mask the pred outputs of pos classes
    y_pred_pos = (y_pred - (1 - y_true) * 1e12)  # mask the pred outputs of neg classes
    zeros = torch.zeros_like(y_pred[..., :1])
    y_pred_neg = torch.cat([y_pred_neg, zeros], dim=-1)
    y_pred_pos = torch.cat([y_pred_pos, zeros], dim=-1)
    neg_loss = torch.logsumexp(y_pred_neg, dim=-1)
    pos_loss = torch.logsumexp(y_pred_pos, dim=-1)

    return (neg_loss + pos_loss).mean()
    
import torch.nn as nn
class CaseClassification(nn.Module):
    def __init__(self, class_num,model_path):
        super(CaseClassification, self).__init__()
        self.bert = AutoModel.from_pretrained(model_path)
        self.linear = nn.Linear(in_features=768, out_features=class_num)

    def forward(self, input_ids=None, attention_mask=None, token_type_ids=None, label=None):
        outputs = self.bert(input_ids, attention_mask, token_type_ids)
        pooler_output = outputs['pooler_output']

        logits = self.linear(pooler_output)
        logits = torch.sigmoid(logits)
        if label is not None:
            loss_fn = nn.BCELoss()
            # loss_fn = nn.BCEWithLogitsLoss()
            loss = loss_fn(logits, label)
            # loss = multilabel_cross_entropy(logits, label)
            return loss, logits

        return logits


# load the model and tokenizer
model = CaseClassification(class_num=class_num,model_path=model_path).to(device)
# tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')
# tokenizer = BertTokenizer.from_pretrained(model_path)

# prepare the optimizer and corresponding hyper-parameters

no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
{'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
{'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(model.parameters(), lr=learning_rate)
# optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)


In [ ]:
# label = valid_label.values[:,1:]
# valid_size = valid_data.shape[0]
# valid_case_size = len(valid_label)
# valid_index_dict = dict(zip(valid_index_list,range(len(valid_index_list))))
# print(label.shape)
# print(valid_label.iloc[:,0].values)
# print(valid_index_dict)


In [ ]:



from torch import logit


label = valid_label.values[:,1:]
valid_size = valid_data.shape[0]
valid_case_size = len(valid_label)
valid_index_dict = dict(zip(valid_index_list,range(len(valid_index_list))))

def cal_metrics(pred_choice,target):
    TP,TN,FN,FP = 0,0,0,0
    # TP predict 和 label 同时为1
    TP += ((pred_choice == 1) & (target == 1)).sum()
    # TN predict 和 label 同时为0
    TN += ((pred_choice == 0) & (target == 0)).sum()
    # FN predict 0 label 1
    FN += ((pred_choice == 0) & (target == 1)).sum()
    # FP predict 1 label 0
    FP += ((pred_choice == 1) & (target == 0)).sum()
    p = TP / (TP + FP+0.001)
    r = TP / (TP + FN+0.001)
    F1 = 2 * r * p / (r + p+0.0001)
    acc = (TP + TN) / (TP + TN + FP + FN)
    return acc,p,r,F1

# def get_predict_label(logits,threshold):
#     for i in range(len(logits)):
#         for j in range(len(logits[0])):
#             if logits[i][j]>threshold: logits[i][j] = 1
#             else: logits[i][j] = 0
#     return logits


def train_fn(train_dataloader,optimizer,epoch):

    print_diff = 50
    model.train()
    running_loss = 0.0
    total_precision = 0
    total_recall = 0
    total_f1 = 0
    total_acc = 0  

    with tqdm(train_dataloader) as t:
        for i, data in enumerate(t):
            id,fact, label= data

            # tokenize the data text
            inputs = tokenizer(list(fact), max_length=512, padding=True, truncation=True, return_tensors='pt')
        
            # move data to device
            input_ids = inputs['input_ids'].to(device)
            token_type_ids = inputs['token_type_ids'].to(device)
            attention_mask = inputs['attention_mask'].to(device)
            label = label.to(device)
            # forward and backward propagations
            loss, logits = model(input_ids, attention_mask, token_type_ids, label)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            # print statistics
            running_loss += loss.item()
            logits = logits.cpu()
            # t.set_postfix(loss=loss.item(),min=torch.min(logits),max=torch.max(logits),mean = torch.mean(logits))
            t.set_postfix(loss=loss.item())
        print('Train Loss:',running_loss/len(train_dataloader))



    


@torch.no_grad()
def valid_fn(valid_dataloader,epoch):
    predict=np.zeros((len(valid_index_list),class_num))

    model.eval()
    total_precision = 0
    total_recall = 0
    total_f1 = 0
    total_acc = 0     

    case_precision = 0
    case_recall = 0
    case_f1 = 0
    case_acc = 0  
    running_loss =0
    n=0
    with tqdm(valid_dataloader) as t:
        for i, data in enumerate(t):
            id,fact, c_label= data

            # tokenize the data text
            inputs = tokenizer(list(fact), max_length=512, padding=True, truncation=True, return_tensors='pt')
            # move data to device
            input_ids = inputs['input_ids'].to(device)
            token_type_ids = inputs['token_type_ids'].to(device)
            attention_mask = inputs['attention_mask'].to(device)
            c_label = c_label.to(device)



            # forward and backward propagations
            loss, logits = model(input_ids, attention_mask, token_type_ids, c_label)
            n+=len(id)
            # print statistics
            running_loss += loss.item()
            t.set_postfix(loss=loss.item())

            threshold = 0.5
            logits[logits>threshold] = 1
            logits[logits<=threshold] = 0



            logits=logits.cpu().numpy()
            # for i in range(logits.shape[0]):
            #     c_predict=np.zeros(class_num)
            #     c_predict[np.argmax(logits[i])]=1
            #     logits[i]=c_predict
            
            # 单句标签f1
            c_label = c_label.cpu().numpy()
            for i in range(len(id)):
                idx = int(id[i])
                row_idx = valid_index_dict[idx]
                predict[row_idx] += logits[i]
            


            for i in range(len(logits)):
                acc,p,r,F1 = cal_metrics(logits[i],c_label[i])        
                # print(logits[0],label[1])       
                total_precision+= p
                total_recall+= r
                total_f1+= F1
                total_acc+= acc

        # 案件标签f1
        predict[predict>1] = 1
        for i in range(predict.shape[0]):

            acc,p,r,F1 = cal_metrics(predict[i],label[i]) 
            case_precision+= p
            case_recall+= r
            case_f1+= F1
            case_acc+= acc
        print('Valid Loss:',running_loss/len(valid_dataloader))
            
        print('Epoch %d Sen Valid   acc: %.4f, pre: %.4f, rec: %.4f, f1: %.4f' % (epoch+1,total_acc/valid_size,total_precision/valid_size,total_recall/valid_size,total_f1/valid_size,))
        print('Epoch %d Case Valid   acc: %.4f, pre: %.4f, rec: %.4f, f1: %.4f' % (epoch+1,case_acc/valid_case_size,case_precision/valid_case_size,case_recall/valid_case_size,case_f1/valid_case_size))
  
for epoch in range(epochs):
    train_fn(train_dataloader,optimizer,epoch)
    valid_fn(valid_dataloader,epoch)
    
